In [1]:
#import req lib
import pandas as pd
import json

In [2]:
with open('t20_wc_match_results.json') as f:
    data = json.load(f) #here f is nothing but file pointer
#data, now lets create df from it
#len(data) --> 1 since its completely a single element
df_match = pd.DataFrame(data[0]['matchSummary'])
df_match.head()

team1        team2       winner     margin   ground     matchDate  \
0      Namibia    Sri Lanka      Namibia    55 runs  Geelong  Oct 16, 2022   
1  Netherlands       U.A.E.  Netherlands  3 wickets  Geelong  Oct 16, 2022   
2     Scotland  West Indies     Scotland    42 runs   Hobart  Oct 17, 2022   
3      Ireland     Zimbabwe     Zimbabwe    31 runs   Hobart  Oct 17, 2022   
4      Namibia  Netherlands  Netherlands  5 wickets  Geelong  Oct 18, 2022   

     scorecard  
0  T20I # 1823  
1  T20I # 1825  
2  T20I # 1826  
3  T20I # 1828  
4  T20I # 1830

In [3]:
df_match.shape #45 rows and 7 columns

(45, 7)

In [4]:
#now we r gng to use this scorecard as key so that we can connect to other tables, like foriegn key interms of sql
#so we rename that col
df_match.rename({'scorecard' : 'match_id'},axis = 1,inplace = True)
df_match.head()

team1        team2       winner     margin   ground     matchDate  \
0      Namibia    Sri Lanka      Namibia    55 runs  Geelong  Oct 16, 2022   
1  Netherlands       U.A.E.  Netherlands  3 wickets  Geelong  Oct 16, 2022   
2     Scotland  West Indies     Scotland    42 runs   Hobart  Oct 17, 2022   
3      Ireland     Zimbabwe     Zimbabwe    31 runs   Hobart  Oct 17, 2022   
4      Namibia  Netherlands  Netherlands  5 wickets  Geelong  Oct 18, 2022   

      match_id  
0  T20I # 1823  
1  T20I # 1825  
2  T20I # 1826  
3  T20I # 1828  
4  T20I # 1830

In [5]:
match_ids_dict = {}

for index, row in df_match.iterrows(): # iterrow is func which will go each row one by one
    key1 = row["team1"]+" Vs "+row["team2"]
    key2 = row["team2"]+" Vs "+row["team1"]  #these r two comb thn we have to add this to this dict
    
    match_ids_dict[key1] = row["match_id"]
    match_ids_dict[key2] = row["match_id"]
match_ids_dict

{'Namibia Vs Sri Lanka': 'T20I # 1823',
 'Sri Lanka Vs Namibia': 'T20I # 1823',
 'Netherlands Vs U.A.E.': 'T20I # 1825',
 'U.A.E. Vs Netherlands': 'T20I # 1825',
 'Scotland Vs West Indies': 'T20I # 1826',
 'West Indies Vs Scotland': 'T20I # 1826',
 'Ireland Vs Zimbabwe': 'T20I # 1828',
 'Zimbabwe Vs Ireland': 'T20I # 1828',
 'Namibia Vs Netherlands': 'T20I # 1830',
 'Netherlands Vs Namibia': 'T20I # 1830',
 'Sri Lanka Vs U.A.E.': 'T20I # 1832',
 'U.A.E. Vs Sri Lanka': 'T20I # 1832',
 'Ireland Vs Scotland': 'T20I # 1833',
 'Scotland Vs Ireland': 'T20I # 1833',
 'West Indies Vs Zimbabwe': 'T20I # 1834',
 'Zimbabwe Vs West Indies': 'T20I # 1834',
 'Netherlands Vs Sri Lanka': 'T20I # 1835',
 'Sri Lanka Vs Netherlands': 'T20I # 1835',
 'Namibia Vs U.A.E.': 'T20I # 1836',
 'U.A.E. Vs Namibia': 'T20I # 1836',
 'Ireland Vs West Indies': 'T20I # 1837',
 'West Indies Vs Ireland': 'T20I # 1837',
 'Scotland Vs Zimbabwe': 'T20I # 1838',
 'Zimbabwe Vs Scotland': 'T20I # 1838',
 'Australia Vs New Zea

we done the above step inorder to combine tables here th erelationship bt two tables is match and team so we combined diff combination in this dict
the way we use this dict is to bring this in the next df

***BATTING SUMMARY***
unlike above table , this has multiple json objects so we have to process each as each element inside has batting stats of each match

In [6]:
#example of extend command
a = [1,2] #array 1
b = [3,4] #array 2
a.extend(b)
a #this same process we r dng to the upcoming data

[1, 2, 3, 4]

In [7]:
#lets process batting summary
with open('t20_wc_batting_summary.json') as f:
    data = json.load(f)
    all_records = [] #this acts as the single list as we append 
    for rec in data: #we consider each match as a record    
        all_records.extend(rec['battingSummary']) #for this we will get an element with 11 datas or bating data of single match 
        #our ultimate goal is to make it as a single list in a csv
        #so extend is basically one array which we append with another array
all_records # now all the str elements are in a single array

#lets create a df for this also
df_batting = pd.DataFrame(all_records)
df_batting.head()

match teamInnings  battingPos             batsmanName  \
0  Namibia Vs Sri Lanka     Namibia           1      Michael van Lingen   
1  Namibia Vs Sri Lanka     Namibia           2           Divan la Cock   
2  Namibia Vs Sri Lanka     Namibia           3  Jan Nicol Loftie-Eaton   
3  Namibia Vs Sri Lanka     Namibia           4           Stephan Baard   
4  Namibia Vs Sri Lanka     Namibia           5      Gerhard Erasmus(c)   

                         dismissal runs balls 4s 6s      SR  
0     c Pramod Madushan b Chameera    3     6  0  0   50.00  
1      c Shanaka b Pramod Madushan    9     9  1  0  100.00  
2        c â€ Mendis b Karunaratne   20    12  1  2  166.66  
3  c DM de Silva b Pramod Madushan   26    24  2  0  108.33  
4     c Gunathilaka b PWH de Silva   20    24  0  0   83.33

In [8]:
#now we have to convert this dismissal column into out or notout this is if there some str then the player is out else player is notout
#lets do it by creating new col baseed on this dismissal col using apply method to it

df_batting["out/not_out"] = df_batting.dismissal.apply(lambda x: "out" if len(x)>0 else "not_out")
df_batting.head(11)

match teamInnings  battingPos             batsmanName  \
0   Namibia Vs Sri Lanka     Namibia           1      Michael van Lingen   
1   Namibia Vs Sri Lanka     Namibia           2           Divan la Cock   
2   Namibia Vs Sri Lanka     Namibia           3  Jan Nicol Loftie-Eaton   
3   Namibia Vs Sri Lanka     Namibia           4           Stephan Baard   
4   Namibia Vs Sri Lanka     Namibia           5      Gerhard Erasmus(c)   
5   Namibia Vs Sri Lanka     Namibia           6            Jan Frylinck   
6   Namibia Vs Sri Lanka     Namibia           7             David Wiese   
7   Namibia Vs Sri Lanka     Namibia           8                 JJ Smit   
8   Namibia Vs Sri Lanka   Sri Lanka           1         Pathum Nissanka   
9   Namibia Vs Sri Lanka   Sri Lanka           2         Kusal Mendisâ€    
10  Namibia Vs Sri Lanka   Sri Lanka           3     Dhananjaya de Silva   

                          dismissal runs balls 4s 6s      SR out/not_out  
0      c Pramod Madushan b Chameera    3     6  0  0   50.00         out  
1       c Shanaka b Pramod Madushan    9     9  1  0  100.00         out  
2         c â€ Mendis b Karunaratne   20    12  1  2  166.66         out  
3   c DM de Silva b Pramod Madushan   26    24  2  0  108.33         out  
4      c Gunathilaka b PWH de Silva   20    24  0  0   83.33         out  
5   run out (Gunathilaka/â€ Mendis)   44    28  4  0  157.14         out  
6          c â€ Mendis b Theekshana    0     1  0  0    0.00         out  
7                                     31    16  2  2  193.75     not_out  
8                 c Smit b Shikongo    9    10  1  0   90.00         out  
9                c â€ Green b Wiese    6     6  0  0  100.00         out  
10            c Shikongo b Frylinck   12    11  1  0  109.09         out

In [9]:
#now we drop dismissal col
df_batting.drop(columns=['dismissal'], inplace=True) #if we dont specify inplace it will create new df and will not modify this dataframe
df_batting.head(100)

match teamInnings  battingPos                 batsmanName  \
0   Namibia Vs Sri Lanka     Namibia           1          Michael van Lingen   
1   Namibia Vs Sri Lanka     Namibia           2               Divan la Cock   
2   Namibia Vs Sri Lanka     Namibia           3      Jan Nicol Loftie-Eaton   
3   Namibia Vs Sri Lanka     Namibia           4               Stephan Baard   
4   Namibia Vs Sri Lanka     Namibia           5          Gerhard Erasmus(c)   
..                   ...         ...         ...                         ...   
95   Sri Lanka Vs U.A.E.   Sri Lanka           6            Dasun Shanaka(c)   
96   Sri Lanka Vs U.A.E.   Sri Lanka           7  Wanindu Hasaranga de Silva   
97   Sri Lanka Vs U.A.E.   Sri Lanka           8         Chamika Karunaratne   
98   Sri Lanka Vs U.A.E.   Sri Lanka           9             Pramod Madushan   
99   Sri Lanka Vs U.A.E.   Sri Lanka          10         Dushmantha Chameera   

   runs balls 4s 6s      SR out/not_out  
0     3     6  0  0   50.00         out  
1     9     9  1  0  100.00         out  
2    20    12  1  2  166.66         out  
3    26    24  2  0  108.33         out  
4    20    24  0  0   83.33         out  
..  ...   ... .. ..     ...         ...  
95    0     1  0  0    0.00         out  
96    2     3  0  0   66.66         out  
97    8    11  1  0   72.72         out  
98    1     1  0  0  100.00     not_out  
99    1     1  0  0  100.00     not_out  

[100 rows x 10 columns]

In [10]:
#in batsman name in some places we have special characters so we have to remove it this is also done using apply lambda
df_batting["batsmanName"] = df_batting["batsmanName"].apply(lambda x: x.replace('â€',''))
df_batting["batsmanName"] = df_batting["batsmanName"].apply(lambda x: x.replace('\xa0',''))
df_batting.head()

match teamInnings  battingPos             batsmanName runs  \
0  Namibia Vs Sri Lanka     Namibia           1      Michael van Lingen    3   
1  Namibia Vs Sri Lanka     Namibia           2           Divan la Cock    9   
2  Namibia Vs Sri Lanka     Namibia           3  Jan Nicol Loftie-Eaton   20   
3  Namibia Vs Sri Lanka     Namibia           4           Stephan Baard   26   
4  Namibia Vs Sri Lanka     Namibia           5      Gerhard Erasmus(c)   20   

  balls 4s 6s      SR out/not_out  
0     6  0  0   50.00         out  
1     9  1  0  100.00         out  
2    12  1  2  166.66         out  
3    24  2  0  108.33         out  
4    24  0  0   83.33         out

In [11]:
#now we have to connect the above table with this table , so we have team names in both table that is the only way we can do it
#in one table its like team 1 and team 2 in another table its like match using vs 
#so we could use team1 and team2 and vs inbetween but in some cases team 1 and team 2 r reversed so we have to consider both cases
#so in the above table we have to implement this condition
#so we create a new column in this df called match id

#we r gng to transform match column using map func which is kind of apply
df_batting["match_id"] = df_batting["match"].map(match_ids_dict)
df_batting.head()

match teamInnings  battingPos             batsmanName runs  \
0  Namibia Vs Sri Lanka     Namibia           1      Michael van Lingen    3   
1  Namibia Vs Sri Lanka     Namibia           2           Divan la Cock    9   
2  Namibia Vs Sri Lanka     Namibia           3  Jan Nicol Loftie-Eaton   20   
3  Namibia Vs Sri Lanka     Namibia           4           Stephan Baard   26   
4  Namibia Vs Sri Lanka     Namibia           5      Gerhard Erasmus(c)   20   

  balls 4s 6s      SR out/not_out     match_id  
0     6  0  0   50.00         out  T20I # 1823  
1     9  1  0  100.00         out  T20I # 1823  
2    12  1  2  166.66         out  T20I # 1823  
3    24  2  0  108.33         out  T20I # 1823  
4    24  0  0   83.33         out  T20I # 1823

In [12]:
 #so now we found a way to link this two tables,so to carryout further operations we export this as a csv file
    
df_batting.to_csv("fact_batting_summary.csv", index = False)

BOWLING SUMMARY

In [13]:
with open('t20_wc_bowling_summary.json') as f:
    data = json.load(f)
    all_records = []
    for rec in data:
        all_records.extend(rec['bowlingSummary'])
all_records[:2]

[{'match': 'Namibia Vs Sri Lanka',
  'bowlingTeam': 'Sri Lanka',
  'bowlerName': 'Maheesh Theekshana',
  'overs': '4',
  'maiden': '0',
  'runs': '23',
  'wickets': '1',
  'economy': '5.75',
  '0s': '7',
  '4s': '0',
  '6s': '0',
  'wides': '2',
  'noBalls': '0'},
 {'match': 'Namibia Vs Sri Lanka',
  'bowlingTeam': 'Sri Lanka',
  'bowlerName': 'Dushmantha Chameera',
  'overs': '4',
  'maiden': '0',
  'runs': '39',
  'wickets': '1',
  'economy': '9.75',
  '0s': '6',
  '4s': '3',
  '6s': '1',
  'wides': '2',
  'noBalls': '0'}]

In [14]:
df_bowling = pd.DataFrame(all_records)
print(df_bowling.shape)
df_bowling.head()

(500, 13)


match bowlingTeam                  bowlerName overs maiden  \
0  Namibia Vs Sri Lanka   Sri Lanka          Maheesh Theekshana     4      0   
1  Namibia Vs Sri Lanka   Sri Lanka         Dushmantha Chameera     4      0   
2  Namibia Vs Sri Lanka   Sri Lanka             Pramod Madushan     4      0   
3  Namibia Vs Sri Lanka   Sri Lanka         Chamika Karunaratne     4      0   
4  Namibia Vs Sri Lanka   Sri Lanka  Wanindu Hasaranga de Silva     4      0   

  runs wickets economy 0s 4s 6s wides noBalls  
0   23       1    5.75  7  0  0     2       0  
1   39       1    9.75  6  3  1     2       0  
2   37       2    9.25  6  3  1     0       0  
3   36       1    9.00  7  3  1     1       0  
4   27       1    6.75  8  1  1     0       0

In [15]:
df_bowling['match_id'] = df_bowling['match'].map(match_ids_dict)
df_bowling.head()

match bowlingTeam                  bowlerName overs maiden  \
0  Namibia Vs Sri Lanka   Sri Lanka          Maheesh Theekshana     4      0   
1  Namibia Vs Sri Lanka   Sri Lanka         Dushmantha Chameera     4      0   
2  Namibia Vs Sri Lanka   Sri Lanka             Pramod Madushan     4      0   
3  Namibia Vs Sri Lanka   Sri Lanka         Chamika Karunaratne     4      0   
4  Namibia Vs Sri Lanka   Sri Lanka  Wanindu Hasaranga de Silva     4      0   

  runs wickets economy 0s 4s 6s wides noBalls     match_id  
0   23       1    5.75  7  0  0     2       0  T20I # 1823  
1   39       1    9.75  6  3  1     2       0  T20I # 1823  
2   37       2    9.25  6  3  1     0       0  T20I # 1823  
3   36       1    9.00  7  3  1     1       0  T20I # 1823  
4   27       1    6.75  8  1  1     0       0  T20I # 1823

In [16]:
df_bowling.to_csv('fact_bowling_summary.csv', index = False)

PROCESS PLAYER INFORMATION

In [17]:
with open('t20_wc_player_info.json') as f:
    data = json.load(f)

In [18]:
df_players = pd.DataFrame(data)

print(df_players.shape)
df_players.head(10)

(219, 6)


name       team    battingStyle  \
0      Michael van Lingen    Namibia   Left hand Bat   
1           Divan la Cock    Namibia  Right hand Bat   
2  Jan Nicol Loftie-Eaton    Namibia   Left hand Bat   
3           Stephan Baard    Namibia  Right hand Bat   
4      Gerhard Erasmus(c)    Namibia  Right hand Bat   
5            Jan Frylinck    Namibia   Left hand Bat   
6             David Wiese    Namibia  Right hand Bat   
7                 JJ Smit    Namibia  Right hand Bat   
8         Pathum Nissanka  Sri Lanka  Right hand Bat   
9         Kusal Mendisâ€   Sri Lanka  Right hand Bat   

                 bowlingStyle          playingRole  \
0             Left arm Medium   Bowling Allrounder   
1                    Legbreak       Opening Batter   
2  Right arm Medium, Legbreak               Batter   
3       Right arm Medium fast               Batter   
4          Right arm Offbreak           Allrounder   
5        Left arm Fast medium           Allrounder   
6       Right arm Medium fast           Allrounder   
7        Left arm Medium fast   Bowling Allrounder   
8                                 Top order Batter   
9                    Legbreak  Wicketkeeper Batter   

                                         description  
0                                                     
1                                                     
2                                                     
3                                                     
4                                                     
5                                                     
6  David Wiese joined a marked outflow of South A...  
7                                                     
8                                                     
9  Blessed with a compact technique, an aggressiv...

In [19]:
#CLEANING WEIRD CHARACTERS
df_players['name'] = df_players['name'].apply(lambda x: x.replace('â€', ''))
df_players['name'] = df_players['name'].apply(lambda x: x.replace('†', ''))
df_players['name'] = df_players['name'].apply(lambda x: x.replace('\xa0', ''))
df_players.head(10)

name       team    battingStyle  \
0      Michael van Lingen    Namibia   Left hand Bat   
1           Divan la Cock    Namibia  Right hand Bat   
2  Jan Nicol Loftie-Eaton    Namibia   Left hand Bat   
3           Stephan Baard    Namibia  Right hand Bat   
4      Gerhard Erasmus(c)    Namibia  Right hand Bat   
5            Jan Frylinck    Namibia   Left hand Bat   
6             David Wiese    Namibia  Right hand Bat   
7                 JJ Smit    Namibia  Right hand Bat   
8         Pathum Nissanka  Sri Lanka  Right hand Bat   
9            Kusal Mendis  Sri Lanka  Right hand Bat   

                 bowlingStyle          playingRole  \
0             Left arm Medium   Bowling Allrounder   
1                    Legbreak       Opening Batter   
2  Right arm Medium, Legbreak               Batter   
3       Right arm Medium fast               Batter   
4          Right arm Offbreak           Allrounder   
5        Left arm Fast medium           Allrounder   
6       Right arm Medium fast           Allrounder   
7        Left arm Medium fast   Bowling Allrounder   
8                                 Top order Batter   
9                    Legbreak  Wicketkeeper Batter   

                                         description  
0                                                     
1                                                     
2                                                     
3                                                     
4                                                     
5                                                     
6  David Wiese joined a marked outflow of South A...  
7                                                     
8                                                     
9  Blessed with a compact technique, an aggressiv...

In [20]:
df_players[df_players['team'] == 'India']

name   team    battingStyle  \
127             KL Rahul  India  Right hand Bat   
128      Rohit Sharma(c)  India  Right hand Bat   
129          Virat Kohli  India  Right hand Bat   
130     Suryakumar Yadav  India  Right hand Bat   
131           Axar Patel  India   Left hand Bat   
132        Hardik Pandya  India  Right hand Bat   
133       Dinesh Karthik  India  Right hand Bat   
134  Ravichandran Ashwin  India  Right hand Bat   
135    Bhuvneshwar Kumar  India  Right hand Bat   
136       Arshdeep Singh  India   Left hand Bat   
137       Mohammed Shami  India  Right hand Bat   
192         Deepak Hooda  India  Right hand Bat   
211         Rishabh Pant  India   Left hand Bat   

                             bowlingStyle          playingRole  \
127                                             Opening Batter   
128                    Right arm Offbreak     Top order Batter   
129                      Right arm Medium     Top order Batter   
130  Right arm Medium, Right arm Offbreak               Batter   
131                Slow Left arm Orthodox   Bowling Allrounder   
132                 Right arm Medium fast           Allrounder   
133                    Right arm Offbreak  Wicketkeeper Batter   
134                    Right arm Offbreak   Bowling Allrounder   
135                      Right arm Medium               Bowler   
136                  Left arm Medium fast               Bowler   
137                        Right arm Fast               Bowler   
192                    Right arm Offbreak           Allrounder   
211                                        Wicketkeeper Batter   

                                           description  
127  A tall, elegant right-hand batsman who can kee...  
128  Languid and easy on the eye, Rohit Sharma owne...  
129  India has given to the world many a great cric...  
130  Hard-hitting 360-degree batter Suryakumar Yada...  
131  Left-arm spinner Axar Patel has been increasin...  
132  Hardik Pandya swears by living life king size ...  
133  Not many would forget the sight of Dinesh Kart...  
134  R Ashwin took the tricks and skills he learned...  
135  At the time of his India debut in 2012, Bhuvne...  
136                                                     
137  Mohammed Shami was India's leading fast bowler...  
192  An allrounder who can bat in any position, Dee...  
211  A match-turning, swashbuckling batter-keeper i...

In [21]:
df_players.to_csv('dim_players_no_images.csv', index = False)